In [0]:
#to access the data/files from the drive
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
#path to the 'DataGenerator.py' and the 'DenseNet3D.py' files
%cd /content/drive/My Drive/gitRendu

#PATH variables (don't forget the last '/')
framesPATH = '/content/drive/My Drive/gitRendu/data/frames/' #where are stored the frames
masksPATH = '/content/drive/My Drive/gitRendu/data/masks/' #where are stored the masks
weightsPATH = '/content/drive/My Drive/gitRendu/data/weights/' #where will be saved the model

#training variables
NO_OF_TRAIN_IMAGES = 100 #don't give more training samples than available number of frames
BATCH_SIZE = 1 #batch size
nbrEpochs = 1 #nbr of training epoch
modelName = 'exampleModel' #the name of the model that will be saved
nbrClasses = 4 #total number of classes in the masks (counting '0' as a class)

/content/drive/My Drive/gitRendu


In [0]:
!pip install tensorflow;
!pip install git+https://www.github.com/keras-team/keras-contrib.git

  Cloning https://www.github.com/keras-team/keras-contrib.git to /tmp/pip-req-build-wmclctth
  Running command git clone -q https://www.github.com/keras-team/keras-contrib.git /tmp/pip-req-build-wmclctth
  Created wheel for keras-contrib: filename=keras_contrib-2.0.8-cp36-none-any.whl size=101065 sha256=1aa8fd70fceaf9e6f4725a263775020e7e1e7eff5cca44fc2ae50ac7abd9366f
  Stored in directory: /tmp/pip-ephem-wheel-cache-_pi6neoo/wheels/11/27/c8/4ed56de7b55f4f61244e2dc6ef3cdbaff2692527a2ce6502ba
Successfully built keras-contrib


In [0]:
%load_ext autoreload

%autoreload 2

import numpy as np
from keras import backend as K
from keras.callbacks import ModelCheckpoint
from keras.callbacks import CSVLogger
from keras.callbacks import EarlyStopping
from keras.optimizers import Adam, RMSprop
from keras.models import Sequential
from DataGenerator import DataGenerator
import DenseNet3D
from keras_contrib.applications import densenet

Using TensorFlow backend.


In [0]:
%reload_ext autoreload

In [0]:
from keras import metrics
from DenseNet3D import DenseNet3D_FCN
from DataGenerator import DataGenerator
import os
np.random.seed(1)

#we separate the dataset in training and validation sets
NO_OF_VALID_IMAGES = len(os.listdir(framesPATH)) - NO_OF_TRAIN_IMAGES
a = np.arange(len(os.listdir(framesPATH)))
np.random.shuffle(a)

train_ids = []
valid_ids = []

for i in range(NO_OF_TRAIN_IMAGES):
    train_ids.append(a[i])
    
for i in range(NO_OF_VALID_IMAGES):
    valid_ids.append(a[i+NO_OF_TRAIN_IMAGES])

partition = {}
    
partition['train'] = train_ids
partition['validation'] = valid_ids

In [0]:
def weighted_categorical_crossentropy(weights):
    """
    A weighted version of keras.objectives.categorical_crossentropy
    
    Variables:
        weights: numpy array of shape (C,) where C is the number of classes
    
    Usage:
        weights = np.array([0.5,2,10]) # Class one at 0.5, class 2 twice the normal weights, class 3 10x, i.e. Ratio.
        loss = weighted_categorical_crossentropy(weights)
        model.compile(loss=loss,optimizer='adam')
    """
    
    weights = K.variable(weights)
    
    def loss(y_true, y_pred):
        # scale predictions so that the class probas of each sample sum to 1
        y_pred /= K.sum(y_pred, axis=-1, keepdims=True)
        # clip to prevent NaN's and Inf's
        y_pred = K.clip(y_pred, K.epsilon(), 1 - K.epsilon())
        # calc
        loss = y_true * K.log(y_pred) * weights
        loss = -K.sum(loss, -1)
        return loss
    
    return loss

In [0]:
from random import randint
Mask = []
N = len(os.listdir(masksPATH))
for ID in range(0,N): 
  mask = np.load(masksPATH+ 'mask_' + str(ID) + '.npy')
  Mask = np.append(Mask,mask)

weight = np.unique(Mask, return_counts = True)[1]
weight = np.sum(weight)/weight
print(weight)

[1.01018371e+00 6.64556910e+02 1.02412309e+03 1.31581764e+02]


In [0]:
print(N)

286


In [0]:
 # Parameters
params = {'dim': (80, 128, 32),
          'batch_size': 1,
          'n_classes': nbrClasses,
          'n_channels': 1,
          'shuffle': True}

# Generators
training_generator = DataGenerator(partition['train'], **params)
validation_generator = DataGenerator(partition['validation'], **params)

# Design model
model = DenseNet3D.DenseNet3D_FCN(input_shape=(80, 128, 32, 1),nb_dense_block=4, growth_rate=16,\
                       nb_layers_per_block=3, upsampling_type='upsampling', classes=params["n_classes"], activation='softmax')

opt = RMSprop(lr=0.001, rho=0.9)

checkpoint = ModelCheckpoint(weightsPATH, monitor='METRIC_TO_MONITOR', 
                             verbose=1, save_best_only=True, mode='max')

csv_logger = CSVLogger('./log.out', append=True, separator=';')

for i in range(params["n_classes"]):
  if i + 1 > len(weight):
    weight = np.append(weight, 0)

u_weight = weight.copy()
# u_weight[1:] = 0.5*weight[1:]
print(weight)
model.compile(loss = weighted_categorical_crossentropy(u_weight), optimizer = opt, metrics=['categorical_accuracy'])

callbacks_list = [csv_logger, checkpoint]

# Train model on dataset
model.fit_generator(generator=training_generator,
                    validation_data=validation_generator,epochs= nbrEpochs, steps_per_epoch = (NO_OF_TRAIN_IMAGES//BATCH_SIZE),\
                    validation_steps=(NO_OF_VALID_IMAGES//BATCH_SIZE), callbacks = callbacks_list)

model.save(weightsPATH + modelName)






[1.01018371e+00 6.64556910e+02 1.02412309e+03 1.31581764e+02]


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



Epoch 1/1





100/100 [==============================] - 134s 1s/step - loss: 10.5837 - categorical_accuracy: 0.9340 - val_loss: 1.7773 - val_categorical_accuracy: 0.9625


/usr/local/lib/python3.6/dist-packages/keras/callbacks.py:707: RuntimeWarning: Can save best model only with METRIC_TO_MONITOR available, skipping.
  'skipping.' % (self.monitor), RuntimeWarning)
